# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')

df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [31]:
e_ids = set(df[df.group=='experiment']['id'].unique())
c_ids = set(df[df.group=='control']['id'].unique())
print('this is the num of ids in both groups:',len(eids&cids))

,timestamp,id,group,action
5639,2016-12-14 13:19:00.940677,182994,control,view
5640,2016-12-14 13:20:29.564080,182994,control,click


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [43]:
control = df[df['group'] == 'control']
experimental = df[df['group'] == 'experiment']

c_v_count = sum(control['action'] == 'view') #getting view count
c_c_count = sum(control['action'] == 'click') #getting click count

ex_v_count = sum(experimental['action'] == 'view')
ex_c_count = sum(experimental['action'] == 'click')

c_click_rate = c_c_count/ c_v_count  #calculating the click through rate
e_click_rate = ex_c_count/ ex_v_count #calculating the click through rate

print('Click through rates are: Control:', c_click_rate, 'Experimental:',e_click_rate)


Click through rates are: Control: 0.2797118847539016 Experimental: 0.3097463284379172


In [88]:
from flatiron_stats import p_value_welch_ttest


control['action'] = control['action'].replace('view', 0)
control['action'] = control['action'].replace('click', 1)

experimental['action'] = experimental['action'].replace('view', 0)
experimental['action'] = experimental['action'].replace('click', 1)

p_value_welch_ttest(control['action'], experimental['action'])



<ipython-input-88-06d7fe831bc3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['action'] = control['action'].replace('view', 0)
<ipython-input-88-06d7fe831bc3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['action'] = control['action'].replace('click', 1)
<ipython-input-88-06d7fe831bc3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

0.026743886922199422

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [73]:
expected_clicks = c_click_rate * ex_v_count

expected_clicks


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [76]:
import math
n = ex_v_count
p = c_click_rate

variance = n*p*(1-p)
std = np.sqrt(variance)
std


24.568547907005815

In [77]:

z_score = (ex_c_count - expected_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [83]:
pval = stats.norm.sf(z_score)

pval

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: No it does not, the pval from the verifying results sections is magnitudes lower than the original P-value that I got leading me to believe that I have a committed an error in my original processing. Although both would result in rejecting the null, these are not close enough values for me to feel comfortable that the calcualtions I did
at first were correct.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.